In [1]:
Q1: Should we use a loop with concatenation or a predefined zeros tensor for creation of dropped indices
    

# What we want to do:
#1) Load two models together--> Loading happens through a session which restores variables, etc. 
#2) We wish to just load the encoder variables (pretrained) and add them to our computation for composite model
 

#What we try doing is based on the following logic:
#1) A graph defines all operations and data flow. 
#2) However, given that a graph can be invoked through a session, we will need to first create a session. 
#3) QUes: Can a graph only be invoked through a session

#4) Then we load this graph and import the encoder part of it into our composite graph.
#5) We finally run a session for this composite graph. 



#6) HOWEVER, we need to make sure that the encoder is not trained any further. 
#7) Otherwise, we will continue with just two separate sessions. 


'''
The final procedure:
1) Create session to load pretrained graph. Store the graph. Quite session. We do this only once. A graph has to be declared in advance. 
2) Set the encoder weights to be non trainable. 
3) Create new session to train original model and use encoder as a part of the operation flow. 


'''




#For optimization and graph construction:
https://www.kdnuggets.com/2017/05/how-not-program-tensorflow-graph.html
    
#General overview of graphs
https://www.tensorflow.org/programmers_guide/graphs#visualizing_your_graph
    
#Making non trainable variables
https://stackoverflow.com/questions/37326002/is-it-possible-to-make-a-trainable-variable-not-trainable?rq=1
https://stackoverflow.com/questions/35298326/freeze-some-variables-scopes-in-tensorflow-stop-gradient-vs-passing-variables
    
#Loading multiple graphs:
https://stackoverflow.com/questions/41990014/load-multiple-models-in-tensorflow
    
#Loading trained weights from one graph to another
https://stackoverflow.com/questions/39068703/tensorflow-using-weights-trained-in-one-model-inside-another-different-model?rq=1

SyntaxError: invalid syntax (<ipython-input-1-3887a96143dd>, line 1)

In [2]:
import os
os.chdir('../model/')
import tensorflow as tf
import pickle
import numpy as np
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def get_word_embeddings(filename):
    try:
        with np.load(filename) as data:
            return data["embeddings"]

    except IOError:
        raise MyIOError(filename)
        
def dataset_load(domain_tr_data_path, vocab_path):
    with open(domain_tr_data_path,'r') as p1:
        domain_tr_data = pickle.load(p1)
    with open(vocab_path,'r') as p1:
        vocab = pickle.load(p1)
        
    domain_tr_data = map(lambda x: x[0],domain_tr_data)
    idd_domain_tr_data = map(lambda x: [vocab[word] for word in x], domain_tr_data)
    return idd_domain_tr_data, vocab

In [4]:
domain_name = 'laptop'
domain_tr_data_path = '../data/Final_joint_data_absa//Domains/Laptop/Normal__normal_training_list.pickle'
embeddings_path = '../data/Embeddings/Pruned/np_glove_200d_trimmed.npz'
embeddings_name = 'glove200d'
vocab_path = '../data/vocab_to_id.pkl'


In [5]:
with open(vocab_path,'r') as p1:
        vocab = pickle.load(p1)
word_embeddings_np = get_word_embeddings(embeddings_path)
pad_token = '<PAD>' 
eos_token = '<END>'
PAD = vocab[pad_token]
EOS = vocab[eos_token]

In [6]:
vocab_size = len(vocab)
input_embedding_size = 200
encoder_hidden_units = 50 #100

In [33]:
def dataset_load(domain_tr_data_path, vocab_path):
    with open(domain_tr_data_path,'r') as p1:
        domain_tr_data = pickle.load(p1)
    with open(vocab_path,'r') as p1:
        vocab = pickle.load(p1)
        
    domain_tr_data = map(lambda x: x[0],domain_tr_data)
    idd_domain_tr_data = map(lambda x: [vocab[word] for word in x], domain_tr_data)
    return idd_domain_tr_data

def get_holdout_data(idd_domain_tr_data, num=10):
    '''Simple function to check if encoder is functioning properly'''
    return idd_domain_tr_data[:num]

def batch_modify(inputs, max_sequence_length=None):
    """
    Args:
        inputs:
            list of sentences (integer lists)
        max_sequence_length:
            integer specifying how large should `max_time` dimension be.
            If None, maximum sequence length would be used
    
    Outputs:
        inputs_time_major:
            input sentences transformed into time-major matrix 
            (shape [max_time, batch_size]) padded with 0s
        sequence_lengths:
            batch-sized list of integers specifying amount of active 
            time steps in each input sequence
    """
    
    sequence_lengths = [len(seq) for seq in inputs]
    batch_size = len(inputs)
    
    if max_sequence_length is None:
        max_sequence_length = max(sequence_lengths)
    
    inputs_batch_major = PAD*np.ones(shape=[batch_size, max_sequence_length], dtype=np.int32) # == PAD
    
    for i, seq in enumerate(inputs):
        for j, element in enumerate(seq):
            inputs_batch_major[i, j] = element

    # [batch_size, max_time] -> [max_time, batch_size]
    inputs_time_major = inputs_batch_major.swapaxes(0, 1)

    return inputs_time_major, sequence_lengths

def feed_enc(enc_batch):
    
    encoder_inputs_, encoder_input_lengths_ = batch_modify(enc_batch)
    return {word_ids: encoder_inputs_,
            sequence_lengths: encoder_input_lengths_}
def feed_enc_train(enc_batch):
    encoder_inputs_, encoder_input_lengths_ = batch_modify(enc_batch)
    train_y = np.zeros(shape = [len(enc_batch),1])#sequence_lengths.shape[0]
    return {word_ids: encoder_inputs_,
            sequence_lengths: encoder_input_lengths_,
            actual_vals:train_y}

In [8]:
word_ids = tf.placeholder(shape=(None, None), dtype=tf.int32, name='word_ids')
sequence_lengths = tf.placeholder(shape=(None,), dtype=tf.int32, name='sequence_lengths')
embeddings = tf.Variable(word_embeddings_np, name="word_embeds",dtype=tf.float32, trainable=False)

In [9]:
'''This is just to test again'''


with tf.variable_scope('seq2seq_encoder'):
        encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, word_ids)
        encoder_cell = LSTMCell(encoder_hidden_units)
        ((encoder_fw_outputs,
          encoder_bw_outputs),
         (encoder_fw_final_state,
          encoder_bw_final_state)) = (
            tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell,
                                    cell_bw=encoder_cell,
                                    inputs=encoder_inputs_embedded,
                                    sequence_length= sequence_lengths,
                                    dtype=tf.float32, time_major=True)
            ) 
    
        encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs),2)
    

        encoder_final_state_c = tf.concat(
            (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

        encoder_final_state_h = tf.concat(
            (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

        encoder_final_state = LSTMStateTuple(
            c=encoder_final_state_c,
            h=encoder_final_state_h
            ) #this is useful later

        encoder_concat_rep = tf.stop_gradient(tf.concat([encoder_final_state_c,encoder_final_state_h], 1))



In [10]:
#model_exi
embed_type = "Glove"
model_path = '../results/seq2seq/{}_seq2seqmodel_embeds{}_{}d_{}hiddenunits.ckpt'.format(domain_name,embed_type,input_embedding_size,encoder_hidden_units)

In [11]:
graph = tf.get_default_graph()
config = tf.ConfigProto(device_count={'GPU': 0})
saver = tf.train.Saver()

In [12]:
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

print("Initialized session")
print("loading existing model")
saver.restore(sess, model_path)

Initialized session
loading existing model
INFO:tensorflow:Restoring parameters from ../results/seq2seq/laptop_seq2seqmodel_embedsGlove_200d_50hiddenunits.ckpt


In [34]:
'''Cell to test in format for experiment'''
def restored_model_enc_out(model_exists_already=True):
    f_enc = feed_enc(holdout_data)
    encoder_useful_state = sess.run(encoder_concat_rep, f_enc)
    return encoder_useful_state

In [35]:
idd_data = dataset_load(domain_tr_data_path,vocab_path)
holdout_data = get_holdout_data(idd_data,10)

In [36]:
encoder_useful_state = restored_model_enc_out()

In [37]:
encoder_useful_state

array([[-0.6898806 ,  0.5807226 ,  0.49744806, ...,  0.5166379 ,
        -0.03120399, -0.18058273],
       [-0.4750856 ,  0.721219  ,  0.6492678 , ...,  0.3919891 ,
        -0.02343164, -0.18083476],
       [-0.3480006 ,  0.43519476,  0.2753141 , ...,  0.19185491,
        -0.29673263, -0.18144843],
       ...,
       [ 0.03478556,  0.4519019 ,  0.0941446 , ...,  0.3134694 ,
        -0.06109187,  0.03018548],
       [-0.18481615,  0.31060803,  0.45174283, ...,  0.47246057,
        -0.11575688, -0.19709298],
       [-0.7678361 ,  0.7076941 ,  0.39733076, ...,  0.21106088,
        -0.06787533, -0.23257294]], dtype=float32)

In [57]:
actual_vals = tf.placeholder(shape=(None, 1),name="blabla", dtype=tf.float32)
with tf.variable_scope("linear"):
    v1 = tf.Variable(tf.ones([encoder_concat_rep.shape[-1],1]),name="v1")
    b1 = tf.Variable(tf.ones([1]), name="v2")
    result = tf.add(tf.matmul(encoder_concat_rep,v1),b1)
    prediction = tf.tanh(result)
    cost = tf.reduce_mean(tf.square(prediction - actual_vals))
    training_step = tf.train.GradientDescentOptimizer(20).minimize(cost)
    

In [58]:
def train_restored_model_enc_out(epochs =2,model_exists_already=True):
    sess.run(tf.variables_initializer([v1, b1]))
    for e_ in range(200):
        f_enc = feed_enc_train(holdout_data)
        _, l,pr = sess.run([training_step,cost, prediction],f_enc)
        print("Prediction:{}".format(pr))
        print("Loss: {}".format(l))
        encoder_useful_state = sess.run(encoder_concat_rep, f_enc)
            
        #encoder_useful_state = sess.run(encoder_concat_everything)
    return encoder_useful_state

In [59]:
encoder_after_training = train_restored_model_enc_out()

Prediction:[[0.9999936 ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [0.99999964]
 [1.        ]
 [0.9999999 ]]
Loss: 0.999998688698
Prediction:[[0.9999934]
 [1.       ]
 [1.       ]
 [1.       ]
 [1.       ]
 [1.       ]
 [1.       ]
 [0.9999997]
 [1.       ]
 [0.9999999]]
Loss: 0.999998569489
Prediction:[[0.99999356]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [0.99999976]
 [1.        ]
 [0.99999976]]
Loss: 0.999998688698
Prediction:[[0.99999344]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [0.99999964]
 [1.        ]
 [0.9999999 ]]
Loss: 0.999998569489
Prediction:[[0.9999933]
 [1.       ]
 [1.       ]
 [1.       ]
 [1.       ]
 [1.       ]
 [1.       ]
 [0.9999997]
 [1.       ]
 [0.9999999]]
Loss: 0.999998569489
Prediction:[[0.9999933 ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [0.99999976]
 [1.        ]
 [0.99999976]]
Los

In [60]:
encoder_useful_state

array([[-0.6898806 ,  0.5807226 ,  0.49744806, ...,  0.5166379 ,
        -0.03120399, -0.18058273],
       [-0.4750856 ,  0.721219  ,  0.6492678 , ...,  0.3919891 ,
        -0.02343164, -0.18083476],
       [-0.3480006 ,  0.43519476,  0.2753141 , ...,  0.19185491,
        -0.29673263, -0.18144843],
       ...,
       [ 0.03478556,  0.4519019 ,  0.0941446 , ...,  0.3134694 ,
        -0.06109187,  0.03018548],
       [-0.18481615,  0.31060803,  0.45174283, ...,  0.47246057,
        -0.11575688, -0.19709298],
       [-0.7678361 ,  0.7076941 ,  0.39733076, ...,  0.21106088,
        -0.06787533, -0.23257294]], dtype=float32)

In [27]:
tf.get_default_graph().get_collection('variables')

[<tf.Variable 'word_embeds:0' shape=(7056, 200) dtype=float32_ref>,
 <tf.Variable 'seq2seq_encoder/bidirectional_rnn/fw/lstm_cell/kernel:0' shape=(250, 200) dtype=float32_ref>,
 <tf.Variable 'seq2seq_encoder/bidirectional_rnn/fw/lstm_cell/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'linear/Variable:0' shape=(200, 1) dtype=float32_ref>,
 <tf.Variable 'linear/Variable_1:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'word_embeds_1:0' shape=(7056, 200) dtype=float32_ref>,
 <tf.Variable 'linear_1/Variable:0' shape=(200, 1) dtype=float32_ref>,
 <tf.Variable 'linear_1/Variable_1:0' shape=(1,) dtype=float32_ref>]

In [49]:
encoder_useful_state

array([[-0.6898806 ,  0.5807226 ,  0.49744806, ...,  0.5166379 ,
        -0.03120399, -0.18058273],
       [-0.4750856 ,  0.721219  ,  0.6492678 , ...,  0.3919891 ,
        -0.02343164, -0.18083476],
       [-0.3480006 ,  0.43519476,  0.2753141 , ...,  0.19185491,
        -0.29673263, -0.18144843],
       ...,
       [ 0.03478556,  0.4519019 ,  0.0941446 , ...,  0.3134694 ,
        -0.06109187,  0.03018548],
       [-0.18481615,  0.31060803,  0.45174283, ...,  0.47246057,
        -0.11575688, -0.19709298],
       [-0.7678361 ,  0.7076941 ,  0.39733076, ...,  0.21106088,
        -0.06787533, -0.23257294]], dtype=float32)

In [17]:
model_meta_path = '../results/seq2seq/laptop_seq2seqmodel_embedsGlove_200d_50hiddenunits.ckpt.meta'

In [19]:
enc_saver = tf.train.import_meta_graph(model_meta_path)

In [20]:
enc_graph = tf.get_default_graph()


In [33]:
enc_graph.get_collection('variables')

[<tf.Variable 'word_embeds:0' shape=(7056, 200) dtype=float32_ref>,
 <tf.Variable 'seq2seq_encoder/bidirectional_rnn/fw/lstm_cell/kernel:0' shape=(250, 200) dtype=float32_ref>,
 <tf.Variable 'seq2seq_encoder/bidirectional_rnn/fw/lstm_cell/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'linear/Variable:0' shape=(200, 1) dtype=float32_ref>,
 <tf.Variable 'linear/Variable_1:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'word_embeds:0' shape=(7056, 200) dtype=float32_ref>,
 <tf.Variable 'word_embeds_1:0' shape=(7056, 200) dtype=float32_ref>,
 <tf.Variable 'word_embeds_2:0' shape=(7056, 200) dtype=float32_ref>,
 <tf.Variable 'seq2seq_encoder/bidirectional_rnn/fw/lstm_cell/kernel:0' shape=(250, 200) dtype=float32_ref>,
 <tf.Variable 'seq2seq_encoder/bidirectional_rnn/fw/lstm_cell/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'Variable:0' shape=(100, 7056) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(7056,) dtype=float32_ref>,
 <tf.Variable 'rnn/lstm_cell/kernel:

In [21]:
enc = enc_graph.get_tensor_by_name("seq2seq_encoder:0")

KeyError: "The name 'seq2seq_encoder:0' refers to a Tensor which does not exist. The operation, 'seq2seq_encoder', does not exist in the graph."

In [34]:
encoder_useful_state = restored_model_enc_out()

Initialized session
loading existing model
INFO:tensorflow:Restoring parameters from ../results/seq2seq/laptop_seq2seqmodel_embedsGlove_200d_50hiddenunits.ckpt


NotFoundError: Key linear/Variable not found in checkpoint
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op u'save/RestoreV2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).star

In [15]:
encoder_useful_state

array([[-0.6898806 ,  0.5807226 ,  0.49744806, ...,  0.5166379 ,
        -0.03120399, -0.18058273],
       [-0.4750856 ,  0.721219  ,  0.6492678 , ...,  0.3919891 ,
        -0.02343164, -0.18083476],
       [-0.3480006 ,  0.43519476,  0.2753141 , ...,  0.19185491,
        -0.29673263, -0.18144843],
       ...,
       [ 0.03478556,  0.4519019 ,  0.0941446 , ...,  0.3134694 ,
        -0.06109187,  0.03018548],
       [-0.18481615,  0.31060803,  0.45174283, ...,  0.47246057,
        -0.11575688, -0.19709298],
       [-0.7678361 ,  0.7076941 ,  0.39733076, ...,  0.21106088,
        -0.06787533, -0.23257294]], dtype=float32)

In [12]:
def bridge(self):
    '''This creates a bridge to our seq2seq model
    The bridge simulates removal of words for each given sentence
    self.word_ids
    self.sequence_lengths
    max_seq_length
    '''
    with tf.variable_scope("drop_words"):
        '''Create mask based on max seq length'''
        max_seq_length = word_ids.shape[-1] 
        np_mask_matrix = np.ones((max_seq_length,max_seq_length)) #num drops=n
        a = np.array(range(max_seq_length))
        np_mask_matrix[np.arange(len(a)), a] = 0
        tf_mask_matrix = tf.convert_to_tensor(np_mask_matrix, dtype="bool")
        padding = tf.constant([[0,0],[0,1]],dtype='int32')
        
        #2nd Operation Add a dimension to both input tensors
        resultant_tensor_for_enc = tf.expand_dims(word_ids,0)
        tensor_seq_lengths_for_enc = tf.expand_dims(sequence_lengths, 0)
        #3rd operation -> Make tensor for seq_lengths for dropped indices (they're always 1 less)
        seq_lengths_for_dropped = tf.expand_dims(sequence_lengths-tf.ones(shape=seq_lengths.shape[0],dtype="int32"),0)

        #4th operation looped-> Applying mask matrix and related operations to get
        #resultant tensor and tensor seq lens 
        #THESE ARE APPENDED to (don't know if that's good)
        for drop_index in range(max_seq_length):
            '''Create mask function-> note the drop index changes at each iteration'''
            f = lambda word_seq: tf.boolean_mask(word_seq, tf_mask_matrix[:,drop_index])#,padding,"CONSTANT") 
            '''Apply masking and padding'''
            resultant_tensor_for_enc = tf.concat([resultant_tensor_for_enc,tf.expand_dims(tf.pad(tf.map_fn(f, word_ids), padding, "CONSTANT"),0)],0)
            tensor_seq_lengths_for_enc = tf.concat([tensor_seq_lengths_for_enc, seq_lengths_for_dropped],0)

        #5th operation--> reshape of tensor
        resultant_tensor_for_enc = tf.reshape(resultant_tensor_for_enc,[resultant_tensor_for_enc.shape[0]*resultant_tensor_for_enc.shape[1],resultant_tensor_for_enc.shape[2]])
        tensor_seq_lengths_for_enc = tf.reshape(tensor_seq_lengths_for_enc,[tensor_seq_lengths_for_enc.shape[0]*tensor_seq_lengths_for_enc.shape[1],])

        
def seq2seq_enc(self):
    #@TODO
    '''Replace encoder_inputs and encoder_inputs_length with general self.word_ids and self.sequence_lengths'''
    '''This is the pretrained encoder'''
    with tf.variable_scope('seq2seq_encoder'):
        encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, resultant_tensor_for_enc)
        encoder_cell = LSTMCell(encoder_hidden_units)
        ((encoder_fw_outputs,
          encoder_bw_outputs),
         (encoder_fw_final_state,
          encoder_bw_final_state)) = (
            tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell,
                                    cell_bw=encoder_cell,
                                    inputs=resultant_tensor_for_enc,
                                    sequence_length= tensor_seq_lengths_for_enc,
                                    dtype=tf.float32, time_major=True)
            ) 
    
        encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs),2)
    

        encoder_final_state_c = tf.concat(
            (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

        encoder_final_state_h = tf.concat(
            (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

        encoder_final_state = LSTMStateTuple(
            c=encoder_final_state_c,
            h=encoder_final_state_h
            ) #this is useful later

        encoder_concat_rep = tf.concat([encoder_final_state_c,encoder_final_state_h], 1)

        


In [14]:
with tf.variable_scope("drop_words"):
        '''Create mask based on max seq length'''
        max_seq_length = word_ids.shape[-1] 
        np_mask_matrix = np.ones((max_seq_length,max_seq_length)) #num drops=n
        a = np.array(range(max_seq_length))
        np_mask_matrix[np.arange(len(a)), a] = 0
        tf_mask_matrix = tf.convert_to_tensor(np_mask_matrix, dtype="bool")
        padding = tf.constant([[0,0],[0,1]],dtype='int32')
        
        #2nd Operation Add a dimension to both input tensors
        resultant_tensor_for_enc = tf.expand_dims(word_ids,0)
        tensor_seq_lengths_for_enc = tf.expand_dims(sequence_lengths, 0)
        #3rd operation -> Make tensor for seq_lengths for dropped indices (they're always 1 less)
        seq_lengths_for_dropped = tf.expand_dims(sequence_lengths-tf.ones(shape=seq_lengths.shape[0],dtype="int32"),0)

        #4th operation looped-> Applying mask matrix and related operations to get
        #resultant tensor and tensor seq lens 
        #THESE ARE APPENDED to (don't know if that's good)
        for drop_index in range(max_seq_length):
            '''Create mask function-> note the drop index changes at each iteration'''
            f = lambda word_seq: tf.boolean_mask(word_seq, tf_mask_matrix[:,drop_index])#,padding,"CONSTANT") 
            '''Apply masking and padding'''
            resultant_tensor_for_enc = tf.concat([resultant_tensor_for_enc,tf.expand_dims(tf.pad(tf.map_fn(f, word_ids), padding, "CONSTANT"),0)],0)
            tensor_seq_lengths_for_enc = tf.concat([tensor_seq_lengths_for_enc, seq_lengths_for_dropped],0)

        #5th operation--> reshape of tensor
        resultant_tensor_for_enc = tf.reshape(resultant_tensor_for_enc,[resultant_tensor_for_enc.shape[0]*resultant_tensor_for_enc.shape[1],resultant_tensor_for_enc.shape[2]])
        tensor_seq_lengths_for_enc = tf.reshape(tensor_seq_lengths_for_enc,[tensor_seq_lengths_for_enc.shape[0]*tensor_seq_lengths_for_enc.shape[1],])

        
with tf.variable_scope('seq2seq_encoder'):
        encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, resultant_tensor_for_enc)
        encoder_cell = LSTMCell(encoder_hidden_units)
        ((encoder_fw_outputs,
          encoder_bw_outputs),
         (encoder_fw_final_state,
          encoder_bw_final_state)) = (
            tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell,
                                    cell_bw=encoder_cell,
                                    inputs=resultant_tensor_for_enc,
                                    sequence_length= tensor_seq_lengths_for_enc,
                                    dtype=tf.float32, time_major=True)
            ) 
    
        encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs),2)
    

        encoder_final_state_c = tf.concat(
            (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

        encoder_final_state_h = tf.concat(
            (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

        encoder_final_state = LSTMStateTuple(
            c=encoder_final_state_c,
            h=encoder_final_state_h
            ) #this is useful later

        encoder_concat_rep = tf.concat([encoder_final_state_c,encoder_final_state_h], 1)


NameError: name 'word_ids' is not defined

In [11]:
graph = tf.get_default_graph()
config = tf.ConfigProto(device_count={'GPU': 0})


In [ ]:
graph.

In [20]:

'''INPUTS'''
input_word_seq_tensor = tf.convert_to_tensor(np.array([[0,1,2,3,1,2],[9,10,11,12,13,12]]),dtype='int32')
max_seq_length = 6
enc_dims = 50
seq_lengths = tf.convert_to_tensor(np.array([4,5]),dtype='int32') #shape n



#1st OPERATION-> Create mask based on max seq length
'''Create mask based on max seq length'''
np_mask_matrix = np.ones((max_seq_length,max_seq_length)) #num drops=n
a = np.array(range(max_seq_length))
np_mask_matrix[np.arange(len(a)), a] = 0
tf_mask_matrix = tf.convert_to_tensor(np_mask_matrix, dtype="bool")
padding = tf.constant([[0,0],[0,1]],dtype='int32')

#2nd Operation Add a dimension to both input tensors
resultant_tensor = tf.expand_dims(input_word_seq_tensor,0)
tensor_seq_lengths = tf.expand_dims(seq_lengths, 0)
#3rd operation -> Make tensor for seq_lengths for dropped indices (they're always 1 less)
seq_lengths_for_dropped = tf.expand_dims(seq_lengths-tf.ones(shape=seq_lengths.shape[0],dtype="int32"),0)
#print(seq_lengths_for_dropped)

#def condition(x):
 #   x.shape[0]< max_seq_length
#def body(x):
 #   Copy

#4th operation looped-> Applying mask matrix and related operations to get
#resultant tensor and tensor seq lens 
#THESE ARE APPENDED to (don't know if that's good)
for drop_index in range(max_seq_length):
    '''Create mask function-> note the drop index changes at each iteration'''
    f = lambda word_seq: tf.boolean_mask(word_seq, tf_mask_matrix[:,drop_index])#,padding,"CONSTANT") 
    '''Apply masking and padding'''
    #temp = tf.pad(tf.map_fn(f, input_word_seq_tensor), padding, "CONSTANT") #We apply mask and then pad result to enable concatenation with original 
    #temp = tf.expand_dims(temp,0)
    resultant_tensor = tf.concat([resultant_tensor,tf.expand_dims(tf.pad(tf.map_fn(f, input_word_seq_tensor), padding, "CONSTANT"),0)],0)
    tensor_seq_lengths = tf.concat([tensor_seq_lengths, seq_lengths_for_dropped],0)

#5th operation--> reshape of tensor
resultant_tensor = tf.reshape(resultant_tensor,[resultant_tensor.shape[0]*resultant_tensor.shape[1],resultant_tensor.shape[2]])
tensor_seq_lengths = tf.reshape(tensor_seq_lengths,[tensor_seq_lengths.shape[0]*tensor_seq_lengths.shape[1],])

#6th operation --> feed to seq2seq encoder and reshape output
'''get_encoder_reps returns the batched encoder representations (shape = [(input_max_length+2)*(input_batch_size) , encoder_dims]'''
#x = get_encoder_reps(resultant_tensor,tensor_seq_lengths,dims)
seq2seq_out = tf.reshape(get_encoder_reps(resultant_tensor,tensor_seq_lengths), [max_seq_length+1, input_word_seq_tensor.shape[0], enc_dims])

#Each column corresponds to the vector rep for a sentence
#Each row corresponds to the word representation

#7th operation --> Obtain the subtracted values/opd values for reps
#We now reduce this by doing an op(row_0,row_i) for all columns
'''NOTE : Hard coded here with subtract have to replace with op(row_0, row_i) for all rows i'''
seq2seq_out = tf.subtract(seq2seq_out,seq2seq_out[0,:])[1:,:]
#So final output should be max_seq_len*n_batches*dims --> do transpose if needed
#8th operation
seq2seq_out = tf.transpose(seq2seq_out,perm=[1,0,2]) #convert to n_batches*max_seq_len*dims or n_batches*words*dims





#apply function operation that takes first index and performs op(first_index, slice)
#print(seq2seq_in.eval())
#print(seq2seq_in.shape)
#print(seq2seq_in_lengths.eval())
#print(seq2seq_in_lengths.shape)


NameError: name 'get_encoder_reps' is not defined

In [ ]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    

In [ ]:
'''
1. If we put seq2seq code inside the other code with all variables
2. Train just the seq2seq (cost, decoded outs, etc)
3. Then save this model with all other variables intact
4. When model is restored we are restoring both seq2seq weights as well as the others and the whole complication of 
5. ONLY complication-> How to set other variables as not trainable 
'''